In [1]:
using JSON,Knet
include(Pkg.dir("Knet","data","imagenet.jl"));
include(Pkg.dir("Knet","examples","vgg","vgg.jl"));
#Pkg.update("HDF5")
using JLD

In [2]:
#generates [x,y] pairs where x=[224,224,3,1] and y=1:classnumber
function readJson(filenames)
    _y=Any[]#class label 
    imgUrls=Any[]
    titles=Any[]
    dataDict=Dict{String,Any}()
    #open file and read the data push 
    for class in 1:length(filenames)
        open(string("Data/"filenames[class],".json"),"r") do f
            jsonObj=Any[] #global
            lines=readlines(f)
            for l in lines
                dict2=JSON.parse(l)
                if(get(dict2,"imUrl",nothing)!=nothing)
                    nestedDict=Dict{String,Any}()
                    nestedDict["class"]=class
                    nestedDict["title"]=get(dict2,"title",nothing)
                    nestedDict["imUrl"]=get(dict2,"imUrl",nothing)
                    dataDict[get(dict2,"asin",nothing)]=nestedDict
                end
            end
        end
    end
    return dataDict
end

readJson (generic function with 1 method)

In [33]:
function prepareImg(dict,atype,model)
    averageImage = convert(Array{Float32},model["meta"]["normalization"]["averageImage"])
    imageSize= convert(Array{Int32},(model["meta"]["normalization"]["imageSize"]))
    for  key in keys(dict)
        dict2=get(dict,key,nothing)
        imUrl=get(dict2,"imUrl",nothing)
        try
            image = Main.imgdata(imUrl, averageImage)
            dict2["image"]=image
            dict[key]=dict2
        catch
            delete!(dict,key)
        end 
    end
    println("Image is prepared with dictionary size", length(dict))
    return dict
end

prepareImg (generic function with 1 method)

In [ ]:
function prepareData(model,atype,imgUrls,_y)
   badurl=0
   #vgg = Main.matconvnet(model) #vgg = Main.matconvnet(o[:model])
   averageImage = convert(Array{Float32},model["meta"]["normalization"]["averageImage"])
   imageSize= convert(Array{Int32},(model["meta"]["normalization"]["imageSize"]))
   a,b,c,d=imageSize
   Data=atype(zeros(a,b,c,length(_y)))
   x=Any[]
   data=Any[]
   clean_y=Any[]
    for i in 1:length(imgUrls)
        if(imgUrls[i]!=nothing)
            try
                image = Main.imgdata(imgUrls[i], averageImage);
                #image = convert(atype, image);
                push!(x, image)
                push!(clean_y,_y[i])
                push!(data, [image,_y[i]])
            catch
                 badurl+=1
                 println(imgUrls[i])
                 println("url is not good")
            end
        else
            badurl+=1
            println("url is not included")
        end
        
    end
    return data
end

In [4]:
function main()
    atype=KnetArray{Float32}
    _filenames=["meta_Baby", "meta_Cell_Phones_and_Accessories","meta_Clothing_Shoes_and_Jewelry",
          "meta_Electronics","meta_Grocery_and_Gourmet_Food","meta_Health_and_Personal_Care",
          "meta_Home_and_Kitchen","meta_Musical_Instruments","meta_Pet_Supplies",
          "meta_Sports_and_Outdoors","meta_Tools_and_Home_Improvement","meta_Toys_and_Games"]
    model="imagenet-vgg-verydeep-16" 
    model = Main.matconvnet(model)
    dict=readJson(_filenames)
    dict2=prepareImg(dict,atype,model)
    save("data.jld","data",dict2)
    return dict2
end


main (generic function with 1 method)